## KOFIC 영화진흥위원회 크롤링

In [3]:
## 영화 포스터와 영화 관객 수의 상관관계를 분석하고자 영화의 포스터와 관객 수를 크롤링하려 한다

### Import package 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import time
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import urllib.request

In [5]:
## kofic 영화진흥위원회 url
url = 'http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do'

In [ ]:
## 크롬 드라이버를 이용한 크롤링

driver = webdriver.Chrome('chromedriver') 
driver.get(url)

## 더보기 버튼 클릭
add = driver.find_element_by_xpath('//*[@id="btn_add"]')
add.click()

time.sleep(1)

## 2000년부터 2018년의 영화 데이터 필터

start = driver.find_element_by_name("sPrdtYearS")
start.click()
start.send_keys("2000")

end = driver.find_element_by_name("sPrdtYearE")
end.click()
end.send_keys("2018")
time.sleep(1)

## 제작 상태 : 개봉
release = driver.find_element_by_name("sPrdtStatStr")
release.click()

time.sleep(1)

release2 = driver.find_element_by_xpath('//*[@id="mul_chk_max0"]')
release2.click()

ok = driver.find_element_by_id("layerConfirmChk")
ok.click()

time.sleep(1)

## 장르별 : 멜로/로맨스, 드라마, 성인물 장르를 제외 Why? 멜로/로맨스, 드라마 장로에도 성인 영화가 굉장히 많이 존재하였는데 이 경우는 영화 포스터가 존재하지 않는 경우가 대부분
## 물론 모두 크롤링한 후 영화 포스터가 존재하지 않는 데이터를 필터해도 되나 이 사이트의 경우 필터를 하는데 상당히 오랜 시간이 걸려 모든 영화를 크롤링하기엔 어려움이 있었음

a1 = driver.find_element_by_css_selector('#sGenreStr')
a1.click()

time.sleep(1)

a2 = driver.find_element_by_css_selector('#chkAllChkBox')
a2.click()
a3 =driver.find_element_by_css_selector('#mul_chk_det0')
a3.click()
a4= driver.find_element_by_css_selector('#mul_chk_det3')
a4.click()
a5= driver.find_element_by_css_selector('#mul_chk_det18')
a5.click()
a6= driver.find_element_by_css_selector('#layerConfirmChk > strong')
a6.click()


## 영화구분 : 일반 영화 선택 ( 예술영화, 독립영화 제외 )
a = driver.find_element_by_name("sMulti")
a.click()
b = driver.find_element_by_name("sIndie")
b.click()

## 조회 버튼 클릭
final = driver.find_element_by_xpath('//*[@id="content"]/form[1]/fieldset/section/p/button[1]')
final.click()


lst = [[],[],[]]
n=0


## 위의 과정은 원하는 영화의 종류를 필터하는 과정이고 아래부터 크롤링 시작 ##

## 이 사이트를 크롤링을 하면서 가장 큰 걸림돌은 영화를 하나씩 클릭하면 나오는 창에서 관객의 수와 이미지를 추출하는 방식으로 진행하였는데, 사이트 자체의 속도가 굉장히 느려 데이터가 긁히지 않는 경우가 굉장히 빈번하였다. 
## 따라서 영화를 검색했을 때 새로운 창이 아직 떠있지 않은 상태인 경우는 계속해서 더욱 기다리는 방식으로 코딩을 진행

for j in range(1, 520): ## 총 520 페이지, 즉 5200개의 영화를 크롤링 (이는 후에 영화가 추가될 경우 변동 가능)
    for i in range(1, 10):  ## 한 페이지에 영화가 10개씩 
        index = str(i)

        time.sleep(0.5)
        
        ## movie를 하나씩 선택
        movie = driver.find_element_by_xpath('//*[@id="content"]/table/tbody/tr['+index+']/td[1]/a')
        lst[0].append(movie.text) ## 먼저 영화의 이름부터 기록
        movie.click()
        
        try:
            image = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.layerBox.ui-dialog-content.ui-widget-content > section:nth-child(1) > div.scroll > article > a > img')            
            lst[1].append(image.get_attribute("src")) ## 영화 이미지 url을 추출
            
        except NoSuchElementException:
            lst[1].append("") ## 이미지가 존재하지 않는 영화는 빈칸으로 처리

        ## 관객 수 추출
        time.sleep(0.5)

        custom = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.ui-dialog-titlebar.ui-widget-header.ui-corner-all.ui-helper-clearfix > section > div > ul > li:nth-child(2) > a')
        custom.click()

        time.sleep(1)
        
        try: 
            customer = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.layerBox.ui-dialog-content.ui-widget-content > section:nth-child(2) > div.scroll > section > div > div:nth-child(1) > table > tbody > tr:nth-child(2) > td:nth-child(4)')
            lst[2].append(customer.text) ## 만일 클릭한 영화의 창이 올바르게 나왔다면 영화 관객 수 추출
        
        except NoSuchElementException: ## 아직 클릭한 영화의 창이 나오지 않았으면 계속해서 기다리며 30초 후에도 나오지 않으면 제외
                time.sleep(1)
                try:
                    error = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.layerBox.ui-dialog-content.ui-widget-content > section:nth-child(2) > div > img')
                    time.sleep(10)
                    try: 
                        customer = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.layerBox.ui-dialog-content.ui-widget-content > section:nth-child(2) > div.scroll > section > div > div:nth-child(1) > table > tbody > tr:nth-child(2) > td:nth-child(4)')
                        lst[2].append(customer.text)
                    except NoSuchElementException:
                        time.sleep(10)
                        try :
                            customer = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.layerBox.ui-dialog-content.ui-widget-content > section:nth-child(2) > div.scroll > section > div > div:nth-child(1) > table > tbody > tr:nth-child(2) > td:nth-child(4)')
                            lst[2].append(customer.text)
                        except NoSuchElementException:
                            time.sleep(10)
                            try:
                                customer = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.layerBox.ui-dialog-content.ui-widget-content > section:nth-child(2) > div.scroll > section > div > div:nth-child(1) > table > tbody > tr:nth-child(2) > td:nth-child(4)')
                                lst[2].append(customer.text)
                            except NoSuchElementException:
                                lst[2].append(-1)
                        
                except NoSuchElementException:
                    lst[2].append(0)
                
                
        close = driver.find_element_by_css_selector('body > div.ui-dialog.ui-widget.ui-widget-content.ui-corner-all.ui-draggable.ui-resizable > div.ui-dialog-titlebar.ui-widget-header.ui-corner-all.ui-helper-clearfix > section > div > a.layer_close > img')
        close.click()
        
    k = j % 10
    
    if(n == 0):
        next = driver.find_element_by_css_selector('#content > form:nth-child(8) > p > a:nth-child('+ str(46) +')')
        next.click()
        n=n+1
    
    else:
        if(k != 0):        
            next = driver.find_element_by_css_selector('#content > form:nth-child(8) > p > a:nth-child('+ str(47+k) +')')
            next.click()    
        else:    
            right = driver.find_element_by_css_selector('#content > form:nth-child(8) > p > a.next > img')
            right.click()
